In [1]:
import numpy as np
import itertools
import pandas as pd
import json

from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve, _SigmoidCalibration
from ampligraph.evaluation import evaluate_performance, mr_score, mrr_score, hits_at_n_score, generate_corruptions_for_eval
from sklearn.metrics import brier_score_loss, log_loss, accuracy_score
from scipy.special import expit

from ampligraph.datasets import load_yago39k
from ampligraph.latent_features.models import TransE, ComplEx, DistMult
import types

In [2]:
from generate_corruptions import generate_corruptions, calibration_loss, pos_iso

In [3]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [4]:
X = load_yago39k()

In [5]:
X_valid_pos = X['valid'][X['valid_labels']]
X_valid_neg = X['valid'][~X['valid_labels']]

X_test_pos = X['test'][X['test_labels']]
X_test_neg = X['test'][~X['test_labels']]

In [6]:
losses =  ['self_adversarial', 'pairwise', 'nll', 'multiclass_nll']
models = [TransE, DistMult, ComplEx]

results = []

for m, l in itertools.product(models, losses):
    model = m(batches_count=64, seed=0, epochs=1000, k=100, eta=20,
                   optimizer='adam', optimizer_params={'lr':0.0001},
                   loss=l, verbose=False)

    model.fit(X['train'])
    
    scores = model.predict(X['test'])

    model.calibrate(X_valid_pos, batches_count=10, epochs=1000, positive_base_rate=0.5)
    print("pos", model.calibration_parameters)
    probas1 = model.predict_proba(X['test'])

    model.calibrate(X_valid_pos, X_valid_neg)
    print("pos neg", model.calibration_parameters)
    probas2 = model.predict_proba(X['test'])
    
    val_scores = model.predict(X['valid'])
    ir = IsotonicRegression(out_of_bounds='clip')
    ir.fit(np.squeeze(val_scores).astype(float), (X['valid_labels']).astype(float))
    probas3 = ir.predict(np.squeeze(scores).astype(float))
    
    model.generate_corruptions = types.MethodType(generate_corruptions, model)
    corruptions = model.generate_corruptions(X_valid_pos, batches_count=10, epochs=1000)
    val_pos_scores = np.squeeze(model.predict(X_valid_pos))
    iso_pos = pos_iso(IsotonicRegression(out_of_bounds='clip'), val_pos_scores, corruptions, positive_base_rate=0.5)
    probas4 = iso_pos.predict(np.squeeze(scores).astype(float))

    sc_pos = pos_iso(_SigmoidCalibration(), val_pos_scores, corruptions, positive_base_rate=0.5)
    print("pos sc", sc_pos.a_, sc_pos.b_)
    probas5 = sc_pos.predict(np.squeeze(scores).astype(float))
    
    val_neg_scores = np.squeeze(model.predict(X_valid_neg))
    sc_pos_neg = pos_iso(_SigmoidCalibration(), val_pos_scores, val_neg_scores, positive_base_rate=0.5)
    print("pos neg sc", sc_pos_neg.a_, sc_pos_neg.b_)
    probas6 = sc_pos_neg.predict(np.squeeze(scores).astype(float))
    
    thresholds = {r: np.median(np.sort(val_scores[X['valid'][:, 1] == r])) for r in np.unique(X['valid'][:, 1])}
    thresholds_test = np.vectorize(thresholds.get)(X['test'][:, 1])
    per_relation_acc = accuracy_score(X['test_labels'], scores > thresholds_test)

    acc_uncalib = accuracy_score(X['test_labels'], expit(scores) > 0.5)

    acc1 = accuracy_score(X['test_labels'], probas1 > 0.5)
    acc2 = accuracy_score(X['test_labels'], probas2 > 0.5)
    acc3 = accuracy_score(X['test_labels'], probas3 > 0.5)
    acc4 = accuracy_score(X['test_labels'], probas4 > 0.5)
    acc5 = accuracy_score(X['test_labels'], probas5 > 0.5)
    acc6 = accuracy_score(X['test_labels'], probas6 > 0.5)
    
    filter_triples = np.concatenate((X['train'], X_valid_pos, X_test_pos))
    ranks = evaluate_performance(X_test_pos, 
                                 model=model, 
                                 filter_triples=filter_triples,
                                 use_default_protocol=True, 
                                 verbose=False)

    results.append({
        'model': m.__name__,
        'loss': l,
        'brier_score_scores': brier_score_loss(X['test_labels'], expit(scores)),
        'log_loss_scores': log_loss(X['test_labels'], expit(scores), eps=1e-7),
        'brier_score_probas_pos': brier_score_loss(X['test_labels'], probas1),
        'log_loss_probas_pos': log_loss(X['test_labels'], probas1, eps=1e-7),
        'brier_score_probas_pos_neg': brier_score_loss(X['test_labels'], probas2),
        'log_loss_probas_pos_neg': log_loss(X['test_labels'], probas2, eps=1e-7),
        'brier_score_probas_pos_neg_iso': brier_score_loss(X['test_labels'], probas3),
        'log_loss_probas_pos_neg_iso': log_loss(X['test_labels'], probas3, eps=1e-7),
        'brier_score_probas_pos_iso': brier_score_loss(X['test_labels'], probas4),
        'log_loss_probas_pos_iso': log_loss(X['test_labels'], probas4, eps=1e-7),
        'brier_score_probas_pos_sc': brier_score_loss(X['test_labels'], probas5),
        'log_loss_probas_pos_sc': log_loss(X['test_labels'], probas5, eps=1e-7),
        'brier_score_probas_pos_neg_sc': brier_score_loss(X['test_labels'], probas6),
        'log_loss_probas_pos_neg_sc': log_loss(X['test_labels'], probas6, eps=1e-7),
        'ece_scores': calibration_loss(X['test_labels'], expit(scores)),
        'ece_probas_pos': calibration_loss(X['test_labels'], probas1),
        'ece_probas_pos_neg': calibration_loss(X['test_labels'], probas2),
        'ece_probas_pos_neg_iso': calibration_loss(X['test_labels'], probas3),
        'ece_probas_pos_iso': calibration_loss(X['test_labels'], probas4),
        'ece_probas_pos_sc': calibration_loss(X['test_labels'], probas5),
        'ece_probas_pos_neg_sc': calibration_loss(X['test_labels'], probas6),
        'metrics_mrr': mrr_score(ranks), 
        'metrics_hits@10': hits_at_n_score(ranks, n=10),
        'metrics_mr': mr_score(ranks),
        'accuracy_per_relation': per_relation_acc,
        'accuracy_uncalib': acc_uncalib,
        'accuracy_pos': acc1,
        'accuracy_pos_neg': acc2,
        'accuracy_pos_neg_iso': acc3,
        'accuracy_pos_iso': acc4,
        'accuracy_pos_sc': acc5,
        'accuracy_pos_neg_sc': acc6
    })
        
    print(json.dumps(results[-1], indent=2))

WARNING - From /home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
WARNING - From /home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:429: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
WARNING - From /home/ptabacof/.

/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in true_divide
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in double_scalars
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count


{
  "model": "TransE",
  "loss": "self_adversarial",
  "brier_score_scores": 0.36302252768719817,
  "log_loss_scores": 1.0622060798888528,
  "brier_score_probas_pos": 0.11324827346184063,
  "log_loss_probas_pos": 0.3990180135640079,
  "brier_score_probas_pos_neg": 0.09484856633983454,
  "log_loss_probas_pos_neg": 0.31857378505872125,
  "brier_score_probas_pos_neg_iso": 0.09250361000856487,
  "log_loss_probas_pos_neg_iso": 0.3081948776976417,
  "brier_score_probas_pos_iso": 0.10927081592075297,
  "log_loss_probas_pos_iso": 0.37590272087934945,
  "brier_score_probas_pos_sc": 0.10616221881001453,
  "log_loss_probas_pos_sc": 0.38805080291314337,
  "brier_score_probas_pos_neg_sc": 0.09480421413974688,
  "log_loss_probas_pos_neg_sc": 0.3185774106611587,
  "ece_scores": 0.40830181367935675,
  "ece_probas_pos": 0.11868441263834635,
  "ece_probas_pos_neg": 0.028639741302818367,
  "ece_probas_pos_neg_iso": 1.1555869761689801e-17,
  "ece_probas_pos_iso": 0.09606850604195921,
  "ece_probas_pos_sc"

/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in true_divide
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count


{
  "model": "TransE",
  "loss": "pairwise",
  "brier_score_scores": 0.49772074388347165,
  "log_loss_scores": 4.921945735433863,
  "brier_score_probas_pos": 0.1470969047874042,
  "log_loss_probas_pos": 0.47739234029725996,
  "brier_score_probas_pos_neg": 0.123208328346179,
  "log_loss_probas_pos_neg": 0.44454980222833146,
  "brier_score_probas_pos_neg_iso": 0.10320889810531554,
  "log_loss_probas_pos_neg_iso": 0.3517519332316571,
  "brier_score_probas_pos_iso": 0.11293827956899179,
  "log_loss_probas_pos_iso": 0.39293455449606485,
  "brier_score_probas_pos_sc": 0.13953547628203666,
  "log_loss_probas_pos_sc": 0.4784827552125384,
  "brier_score_probas_pos_neg_sc": 0.12332184709502418,
  "log_loss_probas_pos_neg_sc": 0.4445549083746815,
  "ece_scores": 0.49774829798488207,
  "ece_probas_pos": 0.19326106799546108,
  "ece_probas_pos_neg": 0.14001669688891338,
  "ece_probas_pos_neg_iso": 9.060852426779504e-17,
  "ece_probas_pos_iso": 0.06509857470870574,
  "ece_probas_pos_sc": 0.1328616564

...

{
  "model": "TransE",
  "loss": "multiclass_nll",
  "brier_score_scores": 0.4977946275853237,
  "log_loss_scores": 7.859535452552008,
  "brier_score_probas_pos": 0.12828468669263407,
  "log_loss_probas_pos": 0.4325858789011811,
  "brier_score_probas_pos_neg": 0.11177965364703041,
  "log_loss_probas_pos_neg": 0.3947554223886153,
  "brier_score_probas_pos_neg_iso": 0.10420754263728874,
  "log_loss_probas_pos_neg_iso": 0.3517703548796889,
  "brier_score_probas_pos_iso": 0.11652963893872059,
  "log_loss_probas_pos_iso": 0.44243625898842465,
  "brier_score_probas_pos_sc": 0.12315189059065008,
  "log_loss_probas_pos_sc": 0.4768157164448518,
  "brier_score_probas_pos_neg_sc": 0.11174939021187671,
  "log_loss_probas_pos_neg_sc": 0.3947600928854345,
  "ece_scores": 0.4977949906549647,
  "ece_probas_pos": 0.1396986916244671,
  "ece_probas_pos_neg": 0.07281328837076821,
  "ece_probas_pos_neg_iso": 7.82528164130957e-18,
  "ece_probas_pos_iso": 0.09125392193509929,
  "ece_probas_pos_sc": 0.0954662

/home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/sklearn/calibration.py:453: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/sklearn/calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)


pos sc -0.238725597936733 2.7487726910510495
pos neg sc -0.22040129037500822 3.315367014924353
{
  "model": "DistMult",
  "loss": "multiclass_nll",
  "brier_score_scores": 0.4563010479896885,
  "log_loss_scores": 4.224559624967882,
  "brier_score_probas_pos": 0.17167909333101242,
  "log_loss_probas_pos": 0.5161328252404302,
  "brier_score_probas_pos_neg": 0.12655919205327312,
  "log_loss_probas_pos_neg": 0.40707941447404056,
  "brier_score_probas_pos_neg_iso": 0.11258924225310064,
  "log_loss_probas_pos_neg_iso": 0.3518024669721689,
  "brier_score_probas_pos_iso": 0.13458584661739978,
  "log_loss_probas_pos_iso": 0.40586321438012934,
  "brier_score_probas_pos_sc": 0.14929025823321715,
  "log_loss_probas_pos_sc": 0.45087411237855207,
  "brier_score_probas_pos_neg_sc": 0.12665846179368587,
  "log_loss_probas_pos_neg_sc": 0.4070846198609235,
  "ece_scores": 0.46467972370886035,
  "ece_probas_pos": 0.15811050456057316,
  "ece_probas_pos_neg": 0.07275065104166667,
  "ece_probas_pos_neg_iso"

/home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/sklearn/calibration.py:453: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/sklearn/calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)


pos sc -0.29024916512395377 2.8758542316570543
pos neg sc -0.2646602843774886 3.4546923104479426
{
  "model": "ComplEx",
  "loss": "multiclass_nll",
  "brier_score_scores": 0.42882663951865674,
  "log_loss_scores": 3.3327317538162293,
  "brier_score_probas_pos": 0.1512251734073152,
  "log_loss_probas_pos": 0.4631039519249512,
  "brier_score_probas_pos_neg": 0.10791405838081022,
  "log_loss_probas_pos_neg": 0.353527897720351,
  "brier_score_probas_pos_neg_iso": 0.10117277542640077,
  "log_loss_probas_pos_neg_iso": 0.3167186081062865,
  "brier_score_probas_pos_iso": 0.12303165845386482,
  "log_loss_probas_pos_iso": 0.3718940172463859,
  "brier_score_probas_pos_sc": 0.12903198374236924,
  "log_loss_probas_pos_sc": 0.3963750608602761,
  "brier_score_probas_pos_neg_sc": 0.10799402360280319,
  "log_loss_probas_pos_neg_sc": 0.3535323817013859,
  "ece_scores": 0.4444651114043369,
  "ece_probas_pos": 0.14252598957348894,
  "ece_probas_pos_neg": 0.07074522203014741,
  "ece_probas_pos_neg_iso": 1

In [7]:
import pandas as pd

In [8]:
def highlight_min(s):
    is_min = s == s.min()
    return ['font-weight: bold' if v else '' for v in is_min]

In [9]:
df = pd.DataFrame(results).set_index(['model', 'loss'])

In [10]:
bs = df[(c for c in df.columns if c.startswith('brier'))]
bs.columns = [c[len("brier_score_"):] for c in bs.columns]
bs.style.apply(highlight_min, axis=1)

In [11]:
ll = df[(c for c in df.columns if c.startswith('log_loss'))]
ll.columns = [c[len("log_loss_"):] for c in ll.columns]
ll.style.apply(highlight_min, axis=1)

In [12]:
print((bs.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'scores', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
 .round(3)
 .to_latex()))

\begin{tabular}{llrrrrr}
\toprule
{} &     model &  scores &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &    TransE &   0.363 &           0.095 &               0.093 &       0.113 &           0.109 \\
1 &  DistMult &   0.284 &           0.081 &               0.079 &       0.091 &           0.089 \\
2 &   ComplEx &   0.264 &           0.089 &               0.084 &       0.095 &           0.095 \\
\bottomrule
\end{tabular}



In [13]:
print(ll.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'scores', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrrr}
\toprule
{} &     model &  scores &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &    TransE &   1.062 &           0.319 &               0.308 &       0.399 &           0.376 \\
1 &  DistMult &   1.043 &           0.279 &               0.266 &       0.309 &           0.308 \\
2 &   ComplEx &   1.199 &           0.305 &               0.278 &       0.319 &           0.313 \\
\bottomrule
\end{tabular}



In [14]:
print((acc*100).reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'pos_neg', 'pos_neg_iso', 'pos', 'pos_iso',  'uncalib', 'per_relation']]
 .reset_index(drop=True)
  .round(1)
 .to_latex())

NameError: name 'acc' is not defined

In [ ]:
metrics = df[(c for c in df.columns if c.startswith('metrics'))]
metrics.columns = [c[len("metrics_"):] for c in metrics.columns]
metrics

In [ ]:
def highlight_max(s):
    is_min = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_min]

acc = df[(c for c in df.columns if c.startswith('accuracy'))]
acc.columns = [c[len("accuracy_"):] for c in acc.columns]
acc.style.apply(highlight_max, axis=1)

In [ ]:
df.corr(method='spearman')

In [ ]:
len(np.unique(X['valid'][:, 1]))